# Plot IM Lup: ALMA vs SPHERE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.spatial.transform import Rotation as R
from seaborn.cm import rocket

sqdeg_per_sr = 4 * np.pi**2 / 360.0**2

import helper
plt.style.use([helper.araa_style])

In [ ]:
fname_S = 'data_IMLup/IM_Lup_reducedRob_median_Hband_12.25mas_mod.fits'
fname_A = 'data_IMLup/IMLup_continuum.fits'

In [ ]:
clip = 3.0       # at how many arcsec to crop the image
drange = 1e-3    # dynamic range of the image
bar_in_au = 100. # how many au the size bar in the image has

# get the distance in parsec based on the GAIA parallax

gaia_plx = 6.4174937230476985 * u.mas
dpc = (u.au / np.tan(gaia_plx)).to('pc').value

Rings measured by Avenhaus et al. 2018

In [ ]:
r_HA = [0.58, 0.96, 1.52, 2.10]
inc_HA = [53., 55, 55., 56.]
PA_HA = 325.
hr_HA =  [0.18, 0.18, 0.23, 0.25]

First two rings are measured by Huang et al. 2018. Last one is my by-eye fit of the outer edge using the average inclination of the other rings

In [ ]:
r_JH = [0.743, 0.845, 1.6]
inc_JH = 47.5
PA_JH = [145.2, 143.9, 144.55]

## Load and plot images

In [ ]:
img_S = helper.image(fname_S, clip=clip)
img_A = helper.image(fname_A, clip=clip)

In [ ]:
fig = plt.figure(figsize=(5, 2.3), dpi=300)
gs = fig.add_gridspec(ncols=3, nrows=1, width_ratios=[1, 1, 0.1], hspace=0.05)
axs = [fig.add_subplot(_gs) for _gs in gs]

for ax in axs[:-1]:
    ax.set_aspect('equal')
    ax.set_xlim(clip, -clip)
    ax.set_ylim(-clip, clip)
    ax.set_xlabel('$\Delta$ RA [arcsec]', fontdict=helper.label_font_dict)
    
# plot the two images

for ax, img in zip(axs, [img_S, img_A]):
    # image
    cc = ax.pcolormesh(img.xi, img.yi, np.maximum(img.data.T, drange), norm=LogNorm(drange, 1.0), rasterized=True)
    
    # scale bar
    ax.plot(-0.8 * clip * np.ones(2) + [0, bar_in_au / dpc], -0.82 * clip * np.ones(2), 'w', lw=3)
    ax.text(-0.8 * clip + bar_in_au / dpc / 2, -0.8 * clip, f'{bar_in_au:.0f} au',
            c='w', horizontalalignment='center', verticalalignment='bottom');
    
# plot the rings in SPHERE

for i_ring in range(4):
    helper.plot_ring(axs[0], 
        r_HA[i_ring],
        hr_HA[i_ring] * r_HA[i_ring],
        np.deg2rad(inc_HA[i_ring]),
        np.deg2rad(90+PA_HA),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--',  dashes=(5, 2))

# plot rings in ALMA
    
for i_ring in range(len(r_JH)):
    helper.plot_ring(axs[1], 
        r_JH[i_ring],
        0.0,
        np.deg2rad(inc_JH),
        np.deg2rad(90+PA_JH[i_ring]),
        nphi=50, c='w', alpha=0.25, lw=1, ls='--', dashes=(5, 2))


# adjust spacing
axs[1].yaxis.set_ticklabels([])
fig.subplots_adjust(wspace=0.02)
    
# colorbar
cb=plt.colorbar(cc, cax=axs[-1])
pos = axs[1].get_position()
axs[-1].set_position([pos.x1+0.02, pos.y0, pos.width / 15., pos.height])
cb.set_label(r'$I_\nu / I_\nu^{max}$', fontdict=helper.label_font_dict)

axs[0].set_ylabel('$\Delta$ Dec [arcsec]', fontdict=helper.label_font_dict);

In [ ]:
fig.savefig(helper.output_dir / 'imlup.pdf', transparent=True, bbox_inches='tight')